In [1]:
# coding: utf-8

from karateclub import NetMF, Node2Vec, FeatherNode

import osmnx as ox
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from copy import deepcopy

repo_path = str(Path.home()) + "/dev/GraphRouteOptimizationRL/"
graph_path = repo_path +     "datasets/osmnx/houston_tx_usa_drive_2000_slope.graphml"

output = repo_path + "datasets/embeddings/houston_tx_usa_drive_2000_slope"

G = ox.load_graphml(graph_path)
G = nx.relabel.convert_node_labels_to_integers(G, first_label=0, ordering='default')

In [2]:
G.add_edge(100, 203)
G.edges[100, 203, 0].update({'osmid': np.random.randint(99999999, 199999999), 'oneway': True, 'lanes': '3', 'name': 'dummy road', 'highway': 'primary',
                                              'length': 64.304, 'speed_kph': 64.9, 'travel_time': 3.6, 'grade': 0.01555, 'grade_abs': 0.01555, 'bearing': 42.3, 'direction': 1})

In [3]:
{int(data['direction']): v for u, v, data in G.edges(100, data=True)}

{0: 266, 6: 1014, 2: 442, 1: 203}

In [5]:
G.edges[100, 203, 0]

{'osmid': 161459772,
 'oneway': True,
 'lanes': '3',
 'name': 'dummy road',
 'highway': 'primary',
 'length': 64.304,
 'speed_kph': 64.9,
 'travel_time': 3.6,
 'grade': 0.01555,
 'grade_abs': 0.01555,
 'bearing': 42.3,
 'direction': 1}

In [2]:
from collections import deque
            
def get_observation(u, G, size = 10):
    dq = deque([u])
    visited = set()
    visited.add(u)
    
    while len(dq) != 0:
        v = dq.popleft()
        for i in G.neighbors(v):
            if i not in visited:
                dq.append(i)
                visited.add(i)
            if len(visited) == size + 1:
                H = G.subgraph(list(visited))
                return H, visited
    raise Exception("Size too large")
    

In [4]:
H, observation_set = get_observation(89, G, 20)

In [7]:
G.edges[45, 44, 0]['length']

233.674

In [ ]:
fig, ax = ox.plot_graph(G, show=False, close=False)

In [20]:
model = NetMF(dimensions=8)
print("Fitting")
model.fit(G)

print("Getting embedding")
X = model.get_embedding()

Fitting
Getting embedding


In [21]:
np.save(output + "_netmf_tmp.npy", X)

In [ ]:



model = Node2Vec(dimensions=8, workers=16)
print("Fitting")
model.fit(G)

print("Getting embedding")
X = model.get_embedding()

np.save(output + "_node2vec.npy", X)

model = NetMF(dimensions=8)
print("Fitting")
model.fit(G)

print("Getting embedding")
X = model.get_embedding()

np.save(output + "_netmf.npy", X)


# model = FeatherNode(reduction_dimensions=32)

# print("Fitting")
# model.fit(G, X)

# print("Getting embedding")
# X = model.get_embedding()

# np.save(output + "e_netmf_feather_32d.npy", X)
